# 문장 유사도를 이용한 ChatBot 개발

In [1]:
# 필요한 라이브러리 설치
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

## 데이터 생성

In [2]:
### 챗봇 생성용 데이터 불러오기

import pandas as pd

file_path = '/content/drive/MyDrive/KDT/딥러닝/자연어처리/ChatBotData.csv'

df = pd.read_csv(file_path)
df

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [ ]:
df.isnull().sum()

Q        0
A        0
label    0
dtype: int64

## 문장 임베딩 행렬 생성

In [3]:
import numpy as np

In [4]:
### 필요한 함수 임폴트
from sentence_transformers import SentenceTransformer

In [5]:
### 사전 학습된 한국어 모델 다운로드
model_name = 'ddobokki/klue-roberta-base-nli-sts'
model = SentenceTransformer(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.79k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/576 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Q 칼럼 인덱싱
sentences = df.loc[:, 'Q']
# print(sentences)

# 생성된 임베딩 벡터 저장 --> 리스트 생성
embeddings = []

# for 문 + model.encode 함수 실행 --> 임베딩 벡터 생성
for sent in sentences:
  sent_embedding = model.encode(sent)
  embeddings.append(sent_embedding)

# 결과 확인하기
print(f'임베딩 벡터 갯수 : {len(embeddings)}')
print('-'*80)
print(f'전체 임베딩 벡터 : \n{embeddings}')

임베딩 벡터 갯수 : 11823
--------------------------------------------------------------------------------


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
print(f'전체 임베딩 벡터 중 상위 3개 출력 : {embeddings[:3]}')

전체 임베딩 벡터 중 상위 3개 출력 : [array([-1.38743356e-01,  1.51100559e-02,  5.28436184e-01,  7.21492112e-01,
       -3.91226649e-01, -7.02952206e-01, -8.01476181e-01,  4.91262108e-01,
        2.17923969e-02,  3.80374342e-01, -6.92348778e-01, -1.11402713e-01,
        2.16700718e-01,  3.51305455e-01,  2.36636266e-01,  6.80069029e-01,
        1.71046928e-01, -5.12855232e-01, -1.09594338e-01,  6.90379366e-02,
        2.15588361e-02, -7.99719989e-01, -1.68676987e-01,  6.13967478e-01,
        3.97407740e-01,  4.25234884e-01,  5.24639748e-02,  2.25223616e-01,
       -3.69577527e-01, -3.86964232e-01, -6.03432000e-01,  4.99111861e-01,
       -1.14106417e-01, -1.77366063e-01,  1.42669812e-01, -5.48125327e-01,
        4.89641666e-01, -2.73698658e-01,  5.87849677e-01,  3.24973583e-01,
        5.86491376e-02,  7.93394744e-01, -1.63209900e-01,  6.36754557e-02,
       -2.92038709e-01, -4.99974638e-01,  2.44824648e-01, -4.05068994e-01,
       -2.36140907e-01,  2.71784455e-01,  7.33441591e-01,  1.45673631e-02,
 

In [ ]:
### 생성된 임베딩 벡터의 리스트 데이터 저장

# 필요한 함수 임폴트
import pickle

# 저장 경로 설정
file_path='/content/drive/MyDrive/KDT/딥러닝/자연어처리/chat_embedding.p'

# 저장 -->  with open + pickle.dump()
with open(file_path, 'wb') as f:
    pickle.dump(embeddings, f)

In [6]:
### 생성된 임베딩 벡터의 리스트 데이터 불러오기

# 필요한 함수 임폴트
import pickle

# 저장 경로 설정
file_path='/content/drive/MyDrive/KDT/딥러닝/자연어처리/chat_embedding.p'

# 저장 -->  with open + pickle.dump()
with open(file_path, 'rb') as f:
  loaded_embeddings = pickle.load(f)

# 결과 확인하기 --> 상위 2개만 출력
print(loaded_embeddings[:2])

[array([-1.38743356e-01,  1.51100559e-02,  5.28436184e-01,  7.21492112e-01,
       -3.91226649e-01, -7.02952206e-01, -8.01476181e-01,  4.91262108e-01,
        2.17923969e-02,  3.80374342e-01, -6.92348778e-01, -1.11402713e-01,
        2.16700718e-01,  3.51305455e-01,  2.36636266e-01,  6.80069029e-01,
        1.71046928e-01, -5.12855232e-01, -1.09594338e-01,  6.90379366e-02,
        2.15588361e-02, -7.99719989e-01, -1.68676987e-01,  6.13967478e-01,
        3.97407740e-01,  4.25234884e-01,  5.24639748e-02,  2.25223616e-01,
       -3.69577527e-01, -3.86964232e-01, -6.03432000e-01,  4.99111861e-01,
       -1.14106417e-01, -1.77366063e-01,  1.42669812e-01, -5.48125327e-01,
        4.89641666e-01, -2.73698658e-01,  5.87849677e-01,  3.24973583e-01,
        5.86491376e-02,  7.93394744e-01, -1.63209900e-01,  6.36754557e-02,
       -2.92038709e-01, -4.99974638e-01,  2.44824648e-01, -4.05068994e-01,
       -2.36140907e-01,  2.71784455e-01,  7.33441591e-01,  1.45673631e-02,
       -6.05235100e-01, 

## ChatBot 데이터 생성


*   embedding이라는 새로운 칼럼 생성
*   문장 임베딩 벡터 저장



In [9]:
# 새로운 칼럼 생성, 예상 질문 임베딩 벡터
df.loc[:, 'embedding'] = loaded_embeddings

# 결과 확인
df.head()

,Q,A,label,embedding
0,12시 땡!,하루가 또 가네요.,0,"[-0.13874336, 0.015110056, 0.5284362, 0.721492..."
1,1지망 학교 떨어졌어,위로해 드립니다.,0,"[0.081922516, 0.30836922, 0.27681017, -0.06013..."
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0,"[-0.062958464, 0.05336736, -0.20226358, -0.468..."
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0,"[-0.19992612, 0.10021132, -0.20123269, -0.5343..."
4,PPL 심하네,눈살이 찌푸려지죠.,0,"[-0.070613146, 0.7641356, -0.29270434, -0.3527..."


In [10]:
df.iloc[0, -1]

array([-1.38743356e-01,  1.51100559e-02,  5.28436184e-01,  7.21492112e-01,
       -3.91226649e-01, -7.02952206e-01, -8.01476181e-01,  4.91262108e-01,
        2.17923969e-02,  3.80374342e-01, -6.92348778e-01, -1.11402713e-01,
        2.16700718e-01,  3.51305455e-01,  2.36636266e-01,  6.80069029e-01,
        1.71046928e-01, -5.12855232e-01, -1.09594338e-01,  6.90379366e-02,
        2.15588361e-02, -7.99719989e-01, -1.68676987e-01,  6.13967478e-01,
        3.97407740e-01,  4.25234884e-01,  5.24639748e-02,  2.25223616e-01,
       -3.69577527e-01, -3.86964232e-01, -6.03432000e-01,  4.99111861e-01,
       -1.14106417e-01, -1.77366063e-01,  1.42669812e-01, -5.48125327e-01,
        4.89641666e-01, -2.73698658e-01,  5.87849677e-01,  3.24973583e-01,
        5.86491376e-02,  7.93394744e-01, -1.63209900e-01,  6.36754557e-02,
       -2.92038709e-01, -4.99974638e-01,  2.44824648e-01, -4.05068994e-01,
       -2.36140907e-01,  2.71784455e-01,  7.33441591e-01,  1.45673631e-02,
       -6.05235100e-01,  

## 사용자 정의 함수 생성

In [11]:
### 두 문장의 텍스트 유사도 계산 함수 정의

# 필요한 함수 임폴트
from numpy import dot
from numpy.linalg import norm

# 사용자 정의 함수
def cos_sim(A, B):
    return dot(A, B) / (norm(A)*norm(B))

In [25]:
### 사용자 질문에 대한 응답 구현 함수
'''
1. 사용자의 질문이 들어오면 해당 질문에 대한 문장 임베딩 값을 구한다.
2. 사용자 질문의 문장 임베딩 값과 챗봇 데이터의 임베딩 컬럼(df.loc[:, 'embedding'])에
저장해둔 모든 질문 샘플들의 문장 임베딩 값들을 이용, 코사인 유사도를 계산
--> 코사인 유사도 값을 저장하는 컬럼(df.loc[:, 'score'])을 생성하고 값을 저장
3. 코사인 유사도 값이 가장 높은 질문 샘플을 찾는다.
4. 해당 질문 샘플과 짝이 되는 답변 샘플을 반환한다.
'''

# 사용자 질문 생성
question = '반가워'
#question = '배가 고파요'

# 사용자 질문 --> 임베딩
user_embedding = model.encode(question)

# 코사인 유사도 계산 --> score 칼럼 생성 --> 코사인 유사도 값을 저장
scores = []
for embedding in df.loc[:, 'embedding']:
  score = cos_sim(user_embedding, embedding)
  scores.append(score)

#print(scores)

# score 칼럼 생성 --> 코사인 유사도 값을 저장
df.loc[:, 'score'] = scores
print(df)

print('-'*80)

# score 칼럼 --> 값이 최대인 인덱스 idxmax
idx = df.loc[:, 'score'].idxmax()
print(f'코사인 유사도가 최대값인 index : {idx}')

print('-'*80)

# 코사인 유사도 값이 가장 큰 예상 질문에 대한 답변 추출
answer = df.loc[idx, 'A']
print(f'챗봇의 답변 : {answer}')

                             Q                         A  label  \
0                       12시 땡!                하루가 또 가네요.      0   
1                  1지망 학교 떨어졌어                 위로해 드립니다.      0   
2                 3박4일 놀러가고 싶다               여행은 언제나 좋죠.      0   
3              3박4일 정도 놀러가고 싶다               여행은 언제나 좋죠.      0   
4                      PPL 심하네                눈살이 찌푸려지죠.      0   
...                        ...                       ...    ...   
11818           훔쳐보는 것도 눈치 보임.        티가 나니까 눈치가 보이는 거죠!      2   
11819           훔쳐보는 것도 눈치 보임.             훔쳐보는 거 티나나봐요.      2   
11820              흑기사 해주는 짝남.                    설렜겠어요.      2   
11821  힘든 연애 좋은 연애라는게 무슨 차이일까?  잘 헤어질 수 있는 사이 여부인 거 같아요.      2   
11822               힘들어서 결혼할까봐        도피성 결혼은 하지 않길 바라요.      2   

                                               embedding     score  
0      [-0.13874336, 0.015110056, 0.5284362, 0.721492...  0.094362  
1      [0.081922516, 0.30836922, 0.27681017, -0.06013... 

In [21]:
### 사용자 질문에 대한 응답 구현 함수 정의
def chatbot(question):
  # 사용자 질문 --> 문장 임베딩 벡터 생성
  user_embedding = model.encode(question)
  # 사용자 질문과 예상 질문 간의 코사인 유사도 계산
  scores = []
  for embedding in df.loc[:, 'embedding']:
    score = cos_sim(user_embedding, embedding)
    scores.append(score)
  # score 칼럼 생성 --> 코사인 유사도 값 저장
  df.loc[:, 'score'] = scores
  # 코사인 유사도 값 최대인 인덱스 추출 --> 답변 추출
  idx = df.loc[:, 'score'].idxmax()
  answer = df.loc[idx, 'A']
  return answer

## ChatBot 실행

In [26]:
## 성능 테스트(1)

# 사용자 입력
question = input()

chat = chatbot(question = question)
print(chat)

귀찮아
귀차니즘을 극복 하셔야해요.
